In [10]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, ndiff
import math
pd.set_option('display.max_colwidth', None)

LABELS = ('X','Y', 'Z', 'A', 'B', 'C', 'D', 'E')

def similarity_and_generalization(col1_value, col2_value):

    col1_phons = str(col1_value).split(';')
    col2_phons = str(col2_value).split(';')
    phon_1_values = []
    phon_2_values = []
    similarity_list = ''

    for phons2 in col2_phons:
        phons2 = phons2.split('|')
        for phon2 in phons2:
            phon_2_values.append(phon2)
            for phons1 in col1_phons:
                phons1 = phons1.split('|')
                for phon1 in phons1:
                    phon_1_values.append(phon1)
                    similarity = str(SequenceMatcher(None, phon1, phon2).ratio())
                    similarity_list = similarity_list + similarity + ';'
    
    similarity_list = similarity_list[:-1]
    similarity_list = similarity_list.split(';')
    similarity_list = list(map(float, similarity_list))
    max_index = similarity_list.index(max(similarity_list))
    
    if 0.0 in similarity_list:
        return similarity_list, 'X', 'Y', '', '', ''
    if 1.0 in similarity_list:
        return similarity_list, 'X', 'X', '', '', ''
    else:
        if(max_index > len(phon_1_values) - 1):
            index = math.floor(float((max_index + 1) / len(phon_1_values)))
            comp_phon_1 = phon_1_values[int(max_index - (int(index) * len(phon_1_values)))]
            comp_phon_2 = phon_2_values[int(index) - 1]
            comparison = ndiff(phon_1_values[int(max_index - (int(index) * len(phon_1_values)))],phon_2_values[int(index) - 1])
            g1, g2, comparison_lst= assign_pattern(comparison)
            return similarity_list, g1, g2, comp_phon_1, comp_phon_2, comparison_lst
        else:
            pass
            comp_phon_1 = phon_1_values[max_index]
            comp_phon_2 = phon_2_values[0]
            comparison = ndiff(phon_1_values[max_index], phon_2_values[0])
            g1, g2, comparison_lst = assign_pattern(comparison)
            return similarity_list, g1, g2, comp_phon_1, comp_phon_2, comparison_lst

def assign_pattern(comparison):
    comparison = list(comparison)
    comparison_lst = ';'.join(comparison)
    kind = [entry[0] for entry in comparison]
    char = [entry[2] for entry in comparison]

    pattern1 = []
    pattern2 = []
    count = 0

    for i, symbol in enumerate(kind):
        if symbol == '+':
            pattern2.append(char[i])
        elif symbol == '-':
            pattern1.append(char[i])
        elif symbol == ' ':
            if i < 1:
                pattern1.append(LABELS[count])
                pattern2.append(LABELS[count])
                count+=1
            elif kind[i-1] != symbol:
                pattern1.append(LABELS[count])
                pattern2.append(LABELS[count])
                count+=1
        
    pattern1= ''.join(pattern1)
    pattern2 = ''.join(pattern2)    
    return(pattern1, pattern2, comparison_lst)

df_rtr = pd.read_csv('produced_tables/rtr.csv', sep='\t', lineterminator='\n', encoding = 'utf8', dtype='unicode')

df_rtr[['similarity', 'generalization_1', 'generalization_2', 'sim_phon_1', 'sim_phon_2', 'comparison']] = df_rtr.apply(lambda row: pd.Series(similarity_and_generalization(row['phon_stem_1'], row['phon_stem_2'])), axis=1)
df_rtr.to_csv('produced_tables/rtr.csv', sep='\t', lineterminator='\n', encoding='utf8', index=False)
display(df_rtr)
display(df_rtr.shape)

,rid,fid,lid_1,graph_1,ori_graph_1,lid_2,graph_2,ori_graph_2,cat_1,ori_cat_1,...,phon_aff_1,phon_aff_2,phon_stem_1,phon_stem_2,similarity,generalization_1,generalization_2,sim_phon_1,sim_phon_2,comparison
0,r0,f203978,l89882,abaisse,converts,l89885,abaisser,converts,Nf,converts,...,NaN,NaN,abɛs;abɛs,abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛse;abɛsə;abɛsa;abɛse,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888]",X,X,,,
1,r102585,f203978,l115595,baisser,derif,l89885,abaisser,derif,V,derif,...,NaN,a.,bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛse;bɛsə;bɛsa;bɛse,bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛse;bɛsə;bɛsa;bɛse,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, ...]",X,X,,,
2,r116505,f203978,l115595,baisser,derif,l89885,abaisser,derif,V,derif,...,NaN,a.,bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛse;bɛsə;bɛsa;bɛse,bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛs;bɛse;bɛsə;bɛsa;bɛse,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, ...]",X,X,,,
3,r116242,f203978,l309187,rabaisser,derif,l89885,abaisser,derif,V,derif,...,ʁə.,NaN,bes;bɛs;bɛs;bes;bɛs;bes;bɛs;bes;bese;bɛsə;besa;bese,abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛs;abɛse;abɛsə;abɛsa;abɛse,"[0.5714285714285714, 0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.5, 0.75, 0.5, 0.5, 0.5714285714285714, 0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.5, 0.75, 0.5, 0.5, 0.5714285714285714, 0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.5, 0.75, 0.5, 0.5, 0.5714285714285714, 0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 0.5714285714285714, 

(90844, 53)